# Gallery Example: M/M/1-PS Feedback Queue

This example demonstrates an M/M/1 queueing system with Processor Sharing and feedback:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Exponential
- **Servers**: 1 server
- **Scheduling**: Processor Sharing (PS)
- **Feedback**: Jobs may return to the queue with probability p

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mm1_ps_feedback(p=1/3):
    """Create M/M/1-PS-Feedback queueing model"""
    model = Network('M/M/1-PS-Feedback')
    
    # Block 1: nodes
    source = Source(model, 'Source')
    queue = Queue(model, 'Queue', SchedStrategy.PS)  # Processor Sharing
    sink = Sink(model, 'Sink')
    
    # Block 2: classes
    oclass1 = OpenClass(model, 'Class1')
    
    # Exponential arrivals and service
    source.set_arrival(oclass1, Exp.fit_mean(1))
    queue.set_service(oclass1, Exp.fit_mean(0.5))
    
    # Block 3: topology with feedback (probability p)
    P = model.init_routing_matrix()
    P.add_route(oclass1, source, queue, 1.0)
    P.add_route(oclass1, queue, queue, p)        # p probability to loop back
    P.add_route(oclass1, queue, sink, 1 - p)     # 1-p probability to exit
    model.link(P)
    
    return model

# Create the model with default p=1/3
model = gallery_mm1_ps_feedback()

## About M/M/1-PS Feedback

With feedback probability p = 1/3:
- Mean visits per job = 1 / (1 - 1/3) = 1.5
- External arrival rate = 1
- Effective arrival rate at queue = 1 × 1.5 = 1.5
- Service rate = 2
- Utilization = 1.5 / 2 = 0.75

**Processor Sharing benefits**:
- Fair allocation regardless of job service requirements
- Response time scales linearly with job size
- Good for interactive systems with feedback

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)